In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

from analystcommunity.write_connection_data_warehouse import runQuery, to_sql

In [2]:
df = pd.read_csv('ALL_SAC_tiers.csv')

In [3]:
# Get the current date in the desired format
current_date = datetime.now().strftime('%Y-%m-%d')

# Add the current date as a new column to the dataframe
df['created_at'] = current_date

In [4]:
df

,city,customer_id,tier,Adjust_CLTV,created_at
0,BOG,-255070,10,19.522333,2023-09-18
1,BOG,-255038,2,1468.994012,2023-09-18
2,BOG,-255037,10,109.293851,2023-09-18
3,BOG,-254992,10,90.714203,2023-09-18
4,BOG,-254982,3,830.128603,2023-09-18
...,...,...,...,...,...
109584,CMX,333970164,10,7.786877,2023-09-18
109585,CMX,346900119,10,0.000000,2023-09-18
109586,CMX,347775810,10,0.000000,2023-09-18
109587,CMX,349047164,10,0.000000,2023-09-18


In [5]:
df2 = df.loc[:,['created_at','city','customer_id','tier']]

In [6]:
df2.columns = ['created_at', 'region_code', 'customer_id', 'cltv']

In [7]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 109589 entries, 0 to 109588
Data columns (total 4 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   created_at   109589 non-null  object
 1   region_code  109589 non-null  object
 2   customer_id  109589 non-null  int64 
 3   cltv         109589 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 3.3+ MB


In [8]:
def put_products_demand_estimations(CLTV_df: pd.DataFrame, region_code: str, remove_previous_records: bool = False):
    
    if remove_previous_records:
        del_test_sql = f"""

        DELETE FROM
            lnd_ops.customer_cltv
        WHERE
            region_code = '{region_code}'
        """

        runQuery('ops', del_test_sql)
    
    to_sql(vertical='ops', 
            df_params=CLTV_df,
            table='customer_cltv')

In [9]:
for city in ['BOG','BAQ','MDE','CMX','CWB','SPO','BHZ','VCP']:
    put_products_demand_estimations(df2.loc[df2.region_code == city], city, False)